In [1]:
import sys
sys.path.append('../')
from utility.file_utility import FileUtility
from scipy.sparse import csr_matrix
import os
import tqdm
path = '/mounts/data/proj/asgari/dissertation/datasets/deepbio/taxonomy/ncbi-blast-2.5.0+/bin/'
os.environ['PATH'] += ':'+path
from Bio.Blast.Applications import NcbiblastnCommandline
from Bio.Blast import NCBIXML

In [ ]:
class FastaMarkers2Excall:
    
    def __init__(fasta_file):
        seq_IDS=FileUtility.read_fasta_sequences_ids('/mounts/data/proj/asgari/dissertation/git_repos/16S_datasets/ra/markers/unt_healthy_chi2_relative.fasta')
    
    def lowest_certain_level(results):
        levels_id={'Superkingdom':1,'phylum':1,'class':2,'order':3,'family':4,'genus':5,'species':6}

        species=set([x[0][levels_id['species']] for x in results])
        genuses=set([x[0][levels_id['genus']] for x in results])
        families=set([x[0][levels_id['family']] for x in results])
        orders=set([x[0][levels_id['order']] for x in results])
        classes=set([x[0][levels_id['class']] for x in results])
        phylums=set([x[0][levels_id['phylum']] for x in results])
        Superkingdoms=set([x[0][levels_id['Superkingdom']] for x in results])

        if len(species)==1:
            return ';'.join(results[0][0])
        elif len(genuses)==1:
            return ';'.join(results[0][0][0:6])
        elif len(families)==1:
            return ';'.join(results[0][0][0:5])
        if len(orders)==1:
            return ';'.join(results[0][0][0:4])
        elif len(classes)==1:
            return ';'.join(results[0][0][0:3]) 
        elif len(phylums)==1:
            return ';'.join(results[0][0][0:2]) 
        elif len(Superkingdoms)==1:
            return ';'.join(results[0][0][0:1]) 
        else:
            return False
        


ez_taxa_dict={x.split()[0]:x.split()[1].split(';') for x in FileUtility.load_list('/mounts/data/proj/asgari/dissertation/git_repos/16S_datasets/EZ/raw/eztaxon_id_taxonomy.txt')}

final_results=[]
for idx, (seq, description) in tqdm.tqdm(seq_IDS.items()):
    pval=float(description.split(':')[1])
    if pval<=0.01:
        FileUtility.create_fasta_file('temp.fasta',[seq],['temp'])
        blastx_cline=NcbiblastnCommandline(query='temp.fasta', db="/mounts/data/proj/asgari/dissertation/git_repos/16S_datasets/EZ/raw/eztaxon_qiime_full.fasta", evalue=0.001, outfmt=5, out="temp.xml")
        blastx_cline()
        f=open("temp.xml",'r')
        blast_records = NCBIXML.parse(f)
        flag=False
        score=-1
        alignment_length=-1
        results=[]
        for blast_record in blast_records:
            for alignment in blast_record.alignments:
                for hsp in alignment.hsps:
                    if not flag and score==-1:
                        score=hsp.score
                        alignment_length=hsp.align_length
                        flag=True
                    if hsp.score >= score and hsp.align_length>=alignment_length:
                        results.append((ez_taxa_dict[alignment.hit_id],hsp.expect))
        if len(results)>0:
            res=is_not_too_ambiguous(results)
            if res:
                final_results.append((seq,res+idx[-1],pval))
        else:
            final_results.append((seq,'ZZZNOVEL'+idx[-1],pval))